In [21]:
import numpy as np
import torch
import torch.nn as nn
import modin.pandas as pd
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [22]:
data = pd.read_csv('diabetes.csv')
data.head()


,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [23]:
x = data.iloc[:,:-1].values
y_s = data.iloc[:,-1].values

In [24]:
y_s

array(['positive', 'negative', 'positive', 'negative', 'positive',
       'negative', 'positive', 'negative', 'positive', 'positive',
       'negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'negative', 'positive',
       'negative', 'negative', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'negative', 'negative',
       'negative', 'positive', 'negative', 'negative', 'negative',
       'negative', 'negative', 'positive', 'positive', 'positive',
       'negative', 'negative', 'negative', 'positive', 'negative',
       'positive', 'negative', 'negative', 'positive', 'negative',
       'negative', 'negative', 'negative', 'positive', 'negative',
       'negative', 'positive', 'negative', 'negative', 'negative',
       'negative', 'positive', 'negative', 'negative', 'positive',
       'negative', 'positive', 'negative', 'negative', 'negative',
       'positive', 'negative', 'positive', 'negative', 'negati

In [25]:
y = [1 if i == 'positive' else 0 for i in y_s]
y = np.array(y,dtype = 'float64')

In [26]:
sc = StandardScaler()
x = sc.fit_transform(x)
x = torch.tensor(x)
y = torch.tensor(y)

print(y.shape)

y = y.unsqueeze(1)

print(y.shape)
print(x.shape)

torch.Size([768])
torch.Size([768, 1])
torch.Size([768, 7])


In [27]:
x

tensor([[ 0.6399,  0.8483,  0.1496,  ..., -0.6929,  0.2040,  1.4260],
        [-0.8449, -1.1234, -0.1605,  ..., -0.6929, -0.6844, -0.1907],
        [ 1.2339,  1.9437, -0.2639,  ..., -0.6929, -1.1033, -0.1056],
        ...,
        [ 0.3430,  0.0033,  0.1496,  ...,  0.2796, -0.7352, -0.2758],
        [-0.8449,  0.1598, -0.4707,  ..., -0.6929, -0.2402,  1.1707],
        [-0.8449, -0.8730,  0.0462,  ..., -0.6929, -0.2021, -0.8714]],
       dtype=torch.float64)

In [28]:
class Dataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return len(self.x)

In [29]:
data_set = Dataset(x = x,y = y)
for i in data_set:
    print(i)
train_loader = DataLoader(dataset = data_set, batch_size = 32, shuffle = True)

(tensor([ 0.6399,  0.8483,  0.1496,  0.9073, -0.6929,  0.2040,  1.4260],
       dtype=torch.float64), tensor([1.], dtype=torch.float64))
(tensor([-0.8449, -1.1234, -0.1605,  0.5309, -0.6929, -0.6844, -0.1907],
       dtype=torch.float64), tensor([0.], dtype=torch.float64))
(tensor([ 1.2339,  1.9437, -0.2639, -1.2882, -0.6929, -1.1033, -0.1056],
       dtype=torch.float64), tensor([1.], dtype=torch.float64))
(tensor([-0.8449, -0.9982, -0.1605,  0.1545,  0.1233, -0.4940, -1.0415],
       dtype=torch.float64), tensor([0.], dtype=torch.float64))
(tensor([-1.1419,  0.5041, -1.5047,  0.9073,  0.7658,  1.4097, -0.0205],
       dtype=torch.float64), tensor([1.], dtype=torch.float64))
(tensor([ 0.3430, -0.1532,  0.2530, -1.2882, -0.6929, -0.8113, -0.2758],
       dtype=torch.float64), tensor([0.], dtype=torch.float64))
(tensor([-0.2510, -1.3425, -0.9877,  0.7191,  0.0712, -0.1260, -0.6161],
       dtype=torch.float64), tensor([1.], dtype=torch.float64))
(tensor([ 1.8278, -0.1845, -3.5726, -1.28

In [30]:
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features,5)
        self.fc2 = nn.Linear(5,4)
        self.fc3 = nn.Linear(4,3)
        self.fc4 = nn.Linear(3,output_features)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x1 = self.fc1(x)
        x1 = self.tanh(x1)
        x2 = self.fc2(x1)
        x2 = self.tanh(x2)
        x3 = self.fc3(x2)
        x3 = self.tanh(x3)
        x4 = self.fc4(x3)
        out = self.sigmoid(x4)
        return out
                      

In [31]:
net = Model(7,1)

criterion = torch.nn.BCELoss(reduction = 'mean')
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)

In [33]:
epochs = 200
for i in range (epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        output = net(inputs)
        loss = criterion(output, labels)
        #cleaning the gradients
        optimizer.zero_grad()
        #backpropagation
        loss.backward()
        #update weights
        optimizer.step()
            
    output = (output>0.5).float()
    accuracy = (output == labels).float().mean()
    print(f"Epoch: {i}, accuracy:{accuracy},loss: {loss}")
    

Epoch: 0, accuracy:0.75,loss: 0.49721643328666687
Epoch: 1, accuracy:0.75,loss: 0.5011851191520691
Epoch: 2, accuracy:0.75,loss: 0.5012141466140747
Epoch: 3, accuracy:0.71875,loss: 0.48675358295440674
Epoch: 4, accuracy:0.75,loss: 0.5356255769729614
Epoch: 5, accuracy:0.78125,loss: 0.36060091853141785
Epoch: 6, accuracy:0.8125,loss: 0.36464446783065796
Epoch: 7, accuracy:0.65625,loss: 0.6202170848846436
Epoch: 8, accuracy:0.65625,loss: 0.576164186000824
Epoch: 9, accuracy:0.84375,loss: 0.2922494411468506
Epoch: 10, accuracy:0.65625,loss: 0.5299867987632751
Epoch: 11, accuracy:0.78125,loss: 0.3335399031639099
Epoch: 12, accuracy:0.875,loss: 0.37901970744132996
Epoch: 13, accuracy:0.8125,loss: 0.40846526622772217
Epoch: 14, accuracy:0.84375,loss: 0.4644467234611511
Epoch: 15, accuracy:0.8125,loss: 0.42755061388015747
Epoch: 16, accuracy:0.65625,loss: 0.5485607385635376
Epoch: 17, accuracy:0.84375,loss: 0.42359092831611633
Epoch: 18, accuracy:0.90625,loss: 0.3405091166496277
Epoch: 19, ac

Epoch: 162, accuracy:0.8125,loss: 0.45250600576400757
Epoch: 163, accuracy:0.75,loss: 0.4435901641845703
Epoch: 164, accuracy:0.71875,loss: 0.4561258554458618
Epoch: 165, accuracy:0.75,loss: 0.44950056076049805
Epoch: 166, accuracy:0.9375,loss: 0.231845885515213
Epoch: 167, accuracy:0.78125,loss: 0.37224018573760986
Epoch: 168, accuracy:0.96875,loss: 0.1760963648557663
Epoch: 169, accuracy:0.75,loss: 0.4218328595161438
Epoch: 170, accuracy:0.84375,loss: 0.32645344734191895
Epoch: 171, accuracy:0.8125,loss: 0.42333337664604187
Epoch: 172, accuracy:0.9375,loss: 0.23826831579208374
Epoch: 173, accuracy:0.875,loss: 0.4144013226032257
Epoch: 174, accuracy:0.8125,loss: 0.3268025815486908
Epoch: 175, accuracy:0.8125,loss: 0.4761757552623749
Epoch: 176, accuracy:0.84375,loss: 0.27808865904808044
Epoch: 177, accuracy:0.875,loss: 0.2971700429916382
Epoch: 178, accuracy:0.90625,loss: 0.2692585289478302
Epoch: 179, accuracy:0.8125,loss: 0.41777071356773376
Epoch: 180, accuracy:0.78125,loss: 0.3753